# Building the Lakehouse

Sentence Transformers

Ref: https://www.sbert.net/docs/pretrained-models/msmarco-v3.html


In [1]:
import faiss  # Make sure to install FAISS
import numpy as np
import pickle
import torch
import os

# sentence transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('msmarco-distilbert-base-tas-b')
VEC_DIM = 768

/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def embed(text):
    return torch.as_tensor(model.encode(text))

In [5]:
if not os.path.isfile("faiss_index.pkl"):
    string_list = ["This course serves as an introductory overview of the rapidly evolving field of machine learning. \
                    Students will acquire a foundational understanding of the various algorithms, techniques, and applications \
                    that make up this discipline. Topics covered include supervised learning, unsupervised learning, \
                    neural networks, decision trees, and support vector machines. Hands-on labs and assignments will provide \
                    practical experience with Python and machine learning libraries such as scikit-learn and TensorFlow.",
                    
                "This course delves into the life, times, and literary contributions of William Shakespeare. Students will \
                    study a selection of his plays, sonnets, and other works to understand their thematic depth, historical \
                    context, and linguistic richness. Literary theories and critical analysis techniques will be used to dissect \
                    the texts.",
                    
                "This course explores the economic aspects of environmental issues such as pollution, conservation, and \
                    sustainable development. Utilizing economic theories and models, students will analyze the market's impact on \
                    the environment and evaluate policy solutions. Topics include externalities, public goods, cost-benefit \
                    analysis, and environmental ethics.",
                    
                "This course traces the development of modern art from the late 19th century to the contemporary era. \
                    Students will explore the various art movements, such as Impressionism, Cubism, and Abstract Expressionism, \
                    and examine their influence on culture and society. Field trips to art galleries and museums will \
                    complement classroom learning."]

    vector_list = [embed(s) for s in string_list]

    vectors = np.array(vector_list)
    
    print(vectors[0].shape)

    index = faiss.IndexFlatL2(VEC_DIM)

    index.add(vectors)

    with open("faiss_index.pkl", "wb") as f:
        pickle.dump(index, f)

    print("FAISS index created and saved.")
else:
    print("FAISS index found.")

(768,)
FAISS index created and saved.


In [4]:
import pickle
import numpy as np

# Load FAISS index from a pickle file
with open("faiss_index.pkl", "rb") as f:
    loaded_index = pickle.load(f)

print("FAISS index loaded.")

# Create a query vector (same dimensionality as original vectors)
query_vector = embed("I want to learn about environmental issues").reshape(1, -1)

# Perform a search on the loaded index
# The function `search` returns two arrays: distances and indices
# The 'k' in the search function specifies that we want to find the nearest 'k' neighbors
k = 4
distances, indices = loaded_index.search(query_vector, k)

print("Distances:", distances)
print("Indices:", indices)


FAISS index loaded.
Distances: [[36.26663 66.52642 76.54465 80.97684]]
Indices: [[2 3 1 0]]
